In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고가 뜨지 않도록..
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

# 출력 창 청소
from IPython.display import clear_output

import time

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 딥러닝
import tensorflow as tf

# 딥러닝 모델 구조를 정의하는 것
from tensorflow.keras.models import Sequential
# 층 모델(은닉층과 출력층)
from tensorflow.keras.layers import Dense
# 활성화 함수를 정의하는 것
from tensorflow.keras.layers import Activation
# CNN : 커널을 통해 합성곱을 구하는 것. 이미지의 특징을 두드러지게 한다.
from tensorflow.keras.layers import Conv2D
# MaxPooling : 불필요한 부분을 제거하는 역할을 한다
from tensorflow.keras.layers import MaxPool2D
# Flatten : CNN, MaxPooling은 2차원 데이터를 사용하고 출력하기 때문에
# 1차원 데이터로 변하는 것을 사용해야 한다.
from tensorflow.keras.layers import Flatten
# Dropout : 노드의 일부를 on/off 하면서 과적합을 예방한다.
from tensorflow.keras.layers import Dropout
# Embedding : 주어진 단어 데이터를 원핫 인코딩하고 차원축소를 한다.
from tensorflow.keras.layers import Embedding


# 데이터 표준화
from sklearn.preprocessing import StandardScaler
# 문자열을 숫자로 변환
from sklearn.preprocessing import LabelEncoder
# 원핫 인코딩
from tensorflow.keras.utils import to_categorical

# 학습데이터와 검증데이터로 나눈다
from sklearn.model_selection import train_test_split

# K폴드 교차검증
from sklearn.model_selection import KFold

# 저장된 모델을 복원한다.
from tensorflow.keras.models import load_model

# epoch 마다 저장을 하기 위해 사용한다.
from tensorflow.keras.callbacks import ModelCheckpoint
# 조기 중단을 위해 사용한다.
from tensorflow.keras.callbacks import EarlyStopping

# 문장을 잘라준다.
from tensorflow.keras.preprocessing.text import Tokenizer
# 모든 문장의 데이터의 단어 데이터 수를 동일한 수로 맞춰준다.
from tensorflow.keras.preprocessing.sequence import pad_sequences
# 문자열을 단어사전으로 만들어준다.
from tensorflow.keras.preprocessing.text import text_to_word_sequence


# GPU 할당
gpus = tf.config.experimental.list_physical_devices('GPU')
# GPU가 있다면..
if len(gpus) > 0 :
    try :
        for gpu in gpus :
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e :
        print(e)

### 문장 자르기


In [2]:
# 테스트 문자열
text = '해보지 않으면 해낼 수 없다'

In [3]:
# 주어진 문장을 토큰화 한다.
result = text_to_word_sequence(text)
print(f'원본 : {text}')
print(f'토큰화 : {result}')


원본 : 해보지 않으면 해낼 수 없다
토큰화 : ['해보지', '않으면', '해낼', '수', '없다']


### 단어 빈도수

In [4]:
docs = [
    '먼저 텍스트와 각 단어를 나누어 토큰화 합니다',
    '텍스트의 단어로 토큰화 해야 딥러닝에서 인식됩니다',
    '토큰화 한 결과는 딥러닝에서 사용할 수 있다'
]


In [5]:
# 토큰화 함수를 통해 전처리를 한다.
token = Tokenizer()
token.fit_on_texts(docs)


In [6]:
# 단어 빈도수
# 전체에서 어떤 단어가 몇 개씩 나왔는가...
token.word_counts


OrderedDict([('먼저', 1),
             ('텍스트와', 1),
             ('각', 1),
             ('단어를', 1),
             ('나누어', 1),
             ('토큰화', 3),
             ('합니다', 1),
             ('텍스트의', 1),
             ('단어로', 1),
             ('해야', 1),
             ('딥러닝에서', 2),
             ('인식됩니다', 1),
             ('한', 1),
             ('결과는', 1),
             ('사용할', 1),
             ('수', 1),
             ('있다', 1)])

In [7]:
# 학습했던 문장의 개수
token.document_count


3

In [8]:
# 각 단어가 몇개의 문장에서 나왔는지.
token.word_docs


defaultdict(int,
            {'합니다': 1,
             '먼저': 1,
             '텍스트와': 1,
             '단어를': 1,
             '각': 1,
             '토큰화': 3,
             '나누어': 1,
             '해야': 1,
             '단어로': 1,
             '텍스트의': 1,
             '인식됩니다': 1,
             '딥러닝에서': 2,
             '사용할': 1,
             '결과는': 1,
             '한': 1,
             '있다': 1,
             '수': 1})

In [9]:
# 각 단어에 부여된 인덱스 (1부터 시작)
token.word_index


{'토큰화': 1,
 '딥러닝에서': 2,
 '먼저': 3,
 '텍스트와': 4,
 '각': 5,
 '단어를': 6,
 '나누어': 7,
 '합니다': 8,
 '텍스트의': 9,
 '단어로': 10,
 '해야': 11,
 '인식됩니다': 12,
 '한': 13,
 '결과는': 14,
 '사용할': 15,
 '수': 16,
 '있다': 17}

### 딥러닝

In [10]:
docs = [
    '너무 재미있네요',
    '최고에요',
    '참 잘 만든 영화에요',
    '추천하고 싶은 영화에요',
    '한 더 보고 싶네요',
    '글쎄요',
    '별로에요',
    '생각보다 지루하네요',
    '연기가 어색해요',
    '재미없어요'
]


In [11]:
# 결과데이터
classes = np.array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0])


In [12]:
# 토큰화
token = Tokenizer()
token.fit_on_texts(docs)


In [13]:
token.word_index # 단어사전

{'영화에요': 1,
 '너무': 2,
 '재미있네요': 3,
 '최고에요': 4,
 '참': 5,
 '잘': 6,
 '만든': 7,
 '추천하고': 8,
 '싶은': 9,
 '한': 10,
 '더': 11,
 '보고': 12,
 '싶네요': 13,
 '글쎄요': 14,
 '별로에요': 15,
 '생각보다': 16,
 '지루하네요': 17,
 '연기가': 18,
 '어색해요': 19,
 '재미없어요': 20}

In [14]:
# 각 문장을 토큰화 시킨 데이터를 단어 인덱스로 변환한다.
X = token.texts_to_sequences(docs)
X

# [2,3] = 너무 재미있네요

[[2, 3],
 [4],
 [5, 6, 7, 1],
 [8, 9, 1],
 [10, 11, 12, 13],
 [14],
 [15],
 [16, 17],
 [18, 19],
 [20]]

In [15]:
# 각 리스트의 데이터의 개수를 최대 개수로 통일한다. 
# 각 리스트의 데이터의 개수를 최대 개수로 통일한다.
padded_x = pad_sequences(X, 4)
padded_x


array([[ 0,  0,  2,  3],
       [ 0,  0,  0,  4],
       [ 5,  6,  7,  1],
       [ 0,  8,  9,  1],
       [10, 11, 12, 13],
       [ 0,  0,  0, 14],
       [ 0,  0,  0, 15],
       [ 0,  0, 16, 17],
       [ 0,  0, 18, 19],
       [ 0,  0,  0, 20]])

In [16]:
# 임베딩 층을 위한 단어의 개수를 구한다.
# 단어사전에서 단어의 번호가 1부터 시작하므로 0부터 인식하는
# 임베딩을 위해 1을 더한다.
word_size = len(token.word_index) + 1
word_size


21

In [19]:
# 학습 모델을 구현한다.
model = Sequential()

# Embedding
# 단어 값들에 원핫 인코딩을 수행한 후 결과와 상관관계가 높은(빈도가 많은) 단어
# 단어 컬럼을 기준으로 지정한 컬럼수 만큼 선택하여 다른 컬럼의 데이터를 압축하는 은닉층
# 첫 번째 : 원핫 인코딩 결과의 단어 수
# 두 번째 : 차원 축소 결과의 단어 수
# 세 번째 : 입력노드의 수
model.add(Embedding(word_size, 3, input_length=4))

# 1차원으로 변경
model.add(Flatten())

# 출력층
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [20]:
# 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam',
              metrics=['accuracy'])


In [21]:
# 학습
model.fit(padded_x, classes, epochs=20)


Epoch 1/20
1/1 [==============================] - 2s 2s/step - loss: 0.6935 - accuracy: 0.4000
Epoch 2/20
1/1 [==============================] - 0s 8ms/step - loss: 0.6922 - accuracy: 0.5000
Epoch 3/20
1/1 [==============================] - 0s 9ms/step - loss: 0.6909 - accuracy: 0.5000
Epoch 4/20
1/1 [==============================] - 0s 8ms/step - loss: 0.6895 - accuracy: 0.5000
Epoch 5/20
1/1 [==============================] - 0s 8ms/step - loss: 0.6882 - accuracy: 0.5000
Epoch 6/20
1/1 [==============================] - 0s 6ms/step - loss: 0.6869 - accuracy: 0.5000
Epoch 7/20
1/1 [==============================] - 0s 7ms/step - loss: 0.6855 - accuracy: 0.6000
Epoch 8/20
1/1 [==============================] - 0s 6ms/step - loss: 0.6842 - accuracy: 0.6000
Epoch 9/20
1/1 [==============================] - 0s 6ms/step - loss: 0.6829 - accuracy: 0.6000
Epoch 10/20
1/1 [==============================] - 0s 5ms/step - loss: 0.6815 - accuracy: 0.6000
Epoch 11/20
1/1 [=======================

In [22]:
a1 = model.evaluate(padded_x, classes)
print(f'손실률 : {a1[0]}')
print(f'정확도 : {a1[1]}')
# 문장이 엄청 많아야 손실률 감소함

1/1 [==============================] - 0s 109ms/step - loss: 0.6667 - accuracy: 1.0000
손실률 : 0.6667443513870239
정확도 : 1.0
